In [ ]:
# import os
# import json
# import pandas as pd
# import numpy as np
# import base64
# import requests
# import asyncio
# import aiohttp
# import nest_asyncio
# import time

# nest_asyncio.apply()
# global all_data_results
# base_directory = "/Users/muhammadarbabarshad/Downloads/Plant_Phenotyping_Datasets/Plant_Phenotyping_Datasets/Plant/Ara2013-Canon"
# all_data = pd.read_csv(base_directory + "/Leaf_counts.csv", header=None)

# def load_image(image_path: str) -> str:
#     """Load image from file and encode it as base64."""
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# api_key = os.getenv("OPENAI_API_KEY")
# model = "gpt-4-turbo"
# url = "https://api.openai.com/v1/chat/completions"
# headers = {
#     "Content-Type": "application/json",
#     "Authorization": f"Bearer {api_key}"
# }

# vision_prompt = """
# Given the image, provide the number of leaves in the picture. Few-shot examples (with image and corresponding outputs) may (or may not) be provided to you. You can learn from those and then predict for the current given image. Be careful, the leafs are convoluted and hard to see so they might look less than anticipated. Give answer in the following JSON format:
# {"leaf_count": number_of_leaves}
# """

# max_requests_per_minute = 80
# request_times = []

# async def get_image_informations(session, inputs: dict) -> dict:
#     """Invoke model with image and prompt."""
#     global request_times
#     current_time = time.time()
    
#     # Remove old request times outside the current time window
#     while request_times and request_times[0] <= current_time - 60:
#         request_times.pop(0)
    
#     if len(request_times) >= max_requests_per_minute:
#         # If the rate limit is exceeded, wait until the oldest request is outside the window
#         time_to_wait = request_times[0] + 60 - current_time
#         await asyncio.sleep(time_to_wait)
    
#     request_times.append(current_time)
    
#     payload = {
#         "model": model,
#         "messages": [
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": vision_prompt},
#                     *inputs['examples'],
#                     {
#                         "type": "image_url",
#                         "image_url": {
#                             "url": f"data:image/jpeg;base64,{inputs['image']}",
#                             "detail": "high"
#                         }
#                     }
#                 ]
#             }
#         ],
#         "max_tokens": 4096
#     }
#     async with session.post(url, headers=headers, json=payload) as response:
#         result = await response.json()
#         print(result)
#         if "choices" in result and result["choices"]:
#             return result
#         else:
#             raise Exception("Unexpected API response format")

# async def process_image(session, i, number_of_shots, all_data_results):
#     try:
#         image_path = base_directory + "/" + all_data[0][i] + "_rgb.png"
#         image_base64 = load_image(image_path)

#         examples = []
#         for j in range(max(0, i - number_of_shots), i):
#             example_image_path = base_directory + "/" + all_data[0][j] + "_rgb.png"
#             example_image_base64 = load_image(example_image_path)
#             examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
#             examples.append({"type": "text", "text": f"Leaf count: {all_data.at[j, 1]}"})

#         result = await get_image_informations(session, {"image": image_base64, "examples": examples})
#         leaf_count = result["choices"][0]['message']['content']
#         all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(leaf_count)['leaf_count']
#     except Exception as e:
#         print(f"Error processing {all_data[0][i]}_rgb.png: {str(e)}")
#         all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'

# async def process_images_for_shots(number_of_shots, all_data_results):
#     async with aiohttp.ClientSession() as session:
#         tasks = []
#         for i in range(10, min(9999999, len(all_data))):
#             task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
#             tasks.append(task)
#         await asyncio.gather(*tasks)

# async def main():
#     all_data_results = all_data.copy(deep=True)
#     all_data_results.columns = all_data_results.columns.map(str)

#     for number_of_shots in [0, 1, 2, 4, 8]:
#         await process_images_for_shots(number_of_shots, all_data_results)
#     all_data_results.to_csv("current_all_data_results_Ara2013-Canon.csv")
#     # print(all_data_results.iloc[9:16])
#     print("done")

# asyncio.run(main())

# modifying for fruits

In [13]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/fruits-images-dataset-object-detection/Train File/Train File'
# FOR API
max_requests_per_half_minute = 20

num_instances_per_class=10

# Define the list of expected class labels
expected_classes = ['Banana', 'Jackfruit', 'Mango', 'Litchi', 'Hog Plum', 'Papaya', 'Grapes', 'Apple', 'Orange', 'Guava']

# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all files in the directory
for filename in os.listdir(base_directory):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        # Extract the label from the filename
        label = filename.split('_')[1].split('.')[0]
        
        # Find the closest match among expected classes using fuzzy matching
        closest_match = process.extractOne(label, expected_classes)
        
        # Get the standardized label (closest match)
        standardized_label = closest_match[0]
        
        # Append the file path and standardized label to the respective lists
        file_paths.append(os.path.join(base_directory, filename))
        labels.append(standardized_label)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)


def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "low"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "low"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("01-Fruit.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

{'id': 'chatcmpl-9THqCXKv8iKkb4P7pPLVsP9Sr2MBo', 'object': 'chat.completion', 'created': 1716767120, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"prediction": "Papaya"}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 208, 'completion_tokens': 7, 'total_tokens': 215}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THqCPbJrXwXGYDdDVs0PJZOJcZI0', 'object': 'chat.completion', 'created': 1716767120, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "Jackfruit"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 208, 'completion_tokens': 10, 'total_tokens': 218}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THqCb5skoOIN9SjmxfWFQI4hq37v', 'object': 'chat.completion', 'created': 1716767120, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "p

In [6]:
all_data_results= pd.read_csv("01-Fruit-high-setting.csv")
all_data_results.head()
#calculate wrong predictions
wrong_predictions = all_data_results[all_data_results['1'] != all_data_results['# of Shots 0']]
print(f"Number of wrong predictions: {len(wrong_predictions)}")
#now printing the samples which are predicted wrong
wrong_predictions.head()

Number of wrong predictions: 2


,Unnamed: 0,0,1,# of Shots 0
64,64,data/fruits-images-dataset-object-detection/Tr...,Papaya,Apple
94,94,data/fruits-images-dataset-object-detection/Tr...,Hog Plum,Orange


In [ ]:
import os;os.getenv("OPENAI_API_KEY")

# 2. drowsy


In [14]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/Drowsy_datset/train'

# FOR API
max_requests_per_half_minute = 20
num_instances_per_class = 50

# Define the list of expected class labels
expected_classes = ['DROWSY', 'NATURAL']

# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir in expected_classes:
        subdir_path = os.path.join(base_directory, subdir)
        # Loop through all files in the subdirectory
        for filename in os.listdir(subdir_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Append the file path and label to the respective lists
                file_paths.append(os.path.join(subdir_path, filename))
                labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("02-Drowsy.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

{'id': 'chatcmpl-9THsMneSPc8CI4LQIE9MPnE4MNCKz', 'object': 'chat.completion', 'created': 1716767254, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "NATURAL"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 347, 'completion_tokens': 11, 'total_tokens': 358}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THsMVw7B1vuApI05vw4SjJoGe807', 'object': 'chat.completion', 'created': 1716767254, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"prediction": "DROWSY"}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 347, 'completion_tokens': 8, 'total_tokens': 355}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THsMlIwPqvJV2YiEDoKiUNKVdCKU', 'object': 'chat.completion', 'created': 1716767254, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"predict

# 3. crop disease

In [15]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/crop-disease'

# FOR API
max_requests_per_half_minute = 20

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Clubroot', 'Canker', 'Anthracnose', 'Blossom End Rot',
       'Verticillium', 'Powdery Mildew', 'Leaf Spots', 'Mosaic Virus',
       'Botrytis', 'Fire Blight', 'Cedar Apple Rust', 'Gray Mold',
       'Fusarium', 'Nematodes', 'Apple Scab', 'Crown Gall', 'Black Spot',
       'Downy Mildew', 'Blight', 'Brown Rot']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("03-Crop.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

{'id': 'chatcmpl-9THumzgtLHHKWlsAGC3c8oLw4JHRb', 'object': 'chat.completion', 'created': 1716767404, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "Clubroot"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 437, 'completion_tokens': 10, 'total_tokens': 447}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THunpZa2m57QXRdfwCRs4cHUBdDB', 'object': 'chat.completion', 'created': 1716767405, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "Apple Scab"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 437, 'completion_tokens': 11, 'total_tokens': 448}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THun0JLdM8rvhMh6IyIS6oq8p9sz', 'object': 'chat.completion', 'created': 1716767405, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content'

# 4. glaucoma

In [16]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = '/Users/muhammadarbabarshad/Documents/Personal Data/GPT4o-with-sakib/data/glaucoma-dataset/Dataset_New/Dataset_New/ACRIMA'

# FOR API
max_requests_per_half_minute = 20

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['glaucoma', 'normal']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0,1, 2, 4]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("04-Glaucoma.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

{'id': 'chatcmpl-9THvhv4uvk6sa3IUUNxjgqi9xHy6K', 'object': 'chat.completion', 'created': 1716767461, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "glaucoma"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 345, 'completion_tokens': 11, 'total_tokens': 356}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THvhrKTziq32gWjUsdz7vAMdWkKd', 'object': 'chat.completion', 'created': 1716767461, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n   "prediction": "glaucoma"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 345, 'completion_tokens': 11, 'total_tokens': 356}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9THvhdT75qAaXcEMCspE3TpU5ZySq', 'object': 'chat.completion', 'created': 1716767461, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content':

# 5. CT

In [17]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = '/Users/muhammadarbabarshad/Documents/Personal Data/GPT4o-with-sakib/data/computed-tomography-ct-of-the-brain/files'

# FOR API
max_requests_per_half_minute = 20

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['aneurysm', 'cancer', 'tumor']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("05-CT.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

{'id': 'chatcmpl-9TI52MrsceOdVEkMOcCChlCMHsSWS', 'object': 'chat.completion', 'created': 1716768040, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"prediction": "aneurysm"}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 350, 'completion_tokens': 8, 'total_tokens': 358}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9TI53SQnV5rgiE5NBavOHejrOXF0Z', 'object': 'chat.completion', 'created': 1716768041, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"prediction": "aneurysm"}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 350, 'completion_tokens': 8, 'total_tokens': 358}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9TI53OPdo03PMAL5j51so5grkOxJh', 'object': 'chat.completion', 'created': 1716768041, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"prediction"

# 6. Captions

In [41]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

import os
import pandas as pd
from sklearn.utils import shuffle

# Set the base directory for images
base_directory_images = '/Users/muhammadarbabarshad/Documents/Personal Data/GPT4o-with-sakib/data/flickr-image-caption/images'

# Set the path for the captions file
captions_file = '/Users/muhammadarbabarshad/Documents/Personal Data/GPT4o-with-sakib/data/flickr-image-caption/captions.txt'

# FOR API
max_requests_per_half_minute = 20
total_samples_to_check = 100

# Read the captions file
captions_df = pd.read_csv(captions_file)

# Sample the required number of rows
sampled_captions = captions_df.sample(n=total_samples_to_check, random_state=42)

# Create empty lists to store file paths and captions
file_paths = []
captions = []

# Iterate over the sampled rows
for index, row in sampled_captions.iterrows():
    image_file = row['image']
    image_caption = row['caption']
    
    # Construct the full file path
    file_path = os.path.join(base_directory_images, image_file)
    
    # Check if the file exists
    if os.path.isfile(file_path):
        file_paths.append(file_path)
        captions.append(image_caption)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: captions})

# Shuffle the data
all_data = shuffle(data, random_state=42).reset_index(drop=True)


def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# vision_prompt = f"""
# Given the image, identify the class of the fruit presented. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
# {{"prediction": "class_name"}}
# Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
# """
vision_prompt = f"""
Given the image, write a short caption for it in a very short single line. Provide your answer in the following JSON format:
{{"prediction": "caption ."}}
Replace "caption" with the appropriate caption based on your analysis of the image. and see i put a full stop (.) at the end of the caption after a space. 
"""

request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("06-Captions.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

KeyboardInterrupt: 

# 7. vqa


In [47]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
import random

nest_asyncio.apply()
global all_data_results

from sklearn.utils import shuffle

# Set the base directory for images
base_directory = '/Users/muhammadarbabarshad/Documents/Personal Data/GPT4o-with-sakib/data/VQA'
images_directory = os.path.join(base_directory, 'images')

# Set the path for the data file
data_file = os.path.join(base_directory, 'data.csv')

# Read the list of possible answers from the answer_space.txt file
with open(os.path.join(base_directory, 'answer_space.txt'), 'r') as f:
    answer_space = f.read().splitlines()

# FOR API
max_requests_per_half_minute = 20
total_samples_to_check = 100

# Read the data file
data_df = pd.read_csv(data_file)

# Sample the required number of rows
sampled_data = data_df.sample(n=total_samples_to_check, random_state=42)
# the answers are comma seperated
sampled_data['answer'] = sampled_data['answer'].apply(lambda x: x.split(',')[0])

# Create a dataframe from the sampled data
data = pd.DataFrame({'file_path': [os.path.join(images_directory, f"{row['image_id']}.png") for _, row in sampled_data.iterrows()],
                     'question': sampled_data['question'],
                     'answer': sampled_data['answer']})

# Shuffle the data
all_data = shuffle(data, random_state=42).reset_index(drop=True)

def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image and question, provide the appropriate answer from the following list of possible answers:
{answer_space}

Provide your answer in the following JSON format:
prediction: "<answer>"
Replace "<answer>" with the appropriate answer from the list above based on your analysis of the image and question. The question is: {{question}}
"""

request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    *inputs['examples'],
                    {"type": "text", "text": vision_prompt.format(question=inputs['question'])},

                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data['file_path'][i]
        question = all_data['question'][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        # random_indices = random.sample(range(num_rows), number_of_shots)
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data['file_path'][j]
            example_image_base64 = load_image(example_image_path)
            example_question = all_data['question'][j]
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f"Question: {example_question}"})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, "answer"]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "question": question, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        prediction_dict = json.loads(prediction)
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = prediction_dict['prediction']

    except Exception as e:
        print(f"Error processing {all_data['file_path'][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:#, 1, 2, 4]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("07-VQA.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

{'id': 'chatcmpl-9TFvi9EmVbeyKJG82JqwOtDClrlGk', 'object': 'chat.completion', 'created': 1716759774, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "chair"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 3096, 'completion_tokens': 9, 'total_tokens': 3105}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9TFvjqZnul6sohAOUkkxovk8RvcLG', 'object': 'chat.completion', 'created': 1716759775, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "prediction": "table"\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 3097, 'completion_tokens': 9, 'total_tokens': 3106}, 'system_fingerprint': 'fp_3196d36131'}
{'id': 'chatcmpl-9TFvjrBTKKsZU1teN7kjzgD6gphBZ', 'object': 'chat.completion', 'created': 1716759775, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n

In [9]:
import pandas as pd
import numpy as np
import os
import json

# Set the base directory for images
base_directory = '/Users/muhammadarbabarshad/Documents/Personal Data/GPT4o-with-sakib/data/VQA'

# Set the path for the data file
data_file = os.path.join(base_directory, 'data.csv')
data_df = pd.read_csv(data_file)

file_read = pd.read_csv("07-VQA-all.csv")

# Merge the data_df with file_read based on the index
merged_df = file_read.merge(data_df[['question', 'answer']], left_index=True, right_index=True)

# Rename the 'answer' column to 'ground_truth'
merged_df = merged_df.rename(columns={'answer': 'ground_truth'})

# Save the updated DataFrame to a new CSV file
merged_df.to_csv("07-VQA-all-with-ground-truth.csv", index=False)